# ESRGAN (Enhanced Super-Resolution Generative Adversarial Networks)
La Red Generadora Adversaria de Súper Resolución Mejorada, o ESRGAN, es un método capaz de generar texturas realistas ampliando la resolución de una imagen. Básicamente es una técnica de aprendizaje automático que utiliza una red adversaria generativa para agrandar la escala de imágenes más pequeñas.

##<h1>PASO 1. Comprobación de GPU actual</h1>
<h4>GPUs recomendados son P100 y T4


In [1]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 418.67, 15079 MiB


## <h1>PASO 2. Clonación de repositorio</h1>


In [2]:
!git clone https://github.com/xinntao/ESRGAN


Cloning into 'ESRGAN'...
remote: Enumerating objects: 216, done.
remote: Total 216 (delta 0), reused 0 (delta 0), pack-reused 216
Receiving objects: 100% (216/216), 24.86 MiB | 44.89 MiB/s, done.
Resolving deltas: 100% (80/80), done.


## <h1>PASO 3. Descargar modelos previamente entrenados</h1>

In [3]:
import gdown
print("Downloading pretrained models")
output1 = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/ESRGAN/models/PPON_D.pth'
output4 = '/content/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True) 
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)  

'/content/ESRGAN/models/PPON_G.pth'

## <h1>Paso 4. Agrega tus propios archivos desde tu PC





In [4]:
from google.colab import drive 
drive.mount('/content/drive')  

Mounted at /content/drive


In [5]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [8]:
 #@markdown  Directorio de archivo origen (ZIP) de Drive y descompresion en FRAME_INPUT_DIR
 ZIP_INPUT_DIR = '/content/drive/My Drive/DAIN_3_OUT.zip' #@param{type:"string"}

 !unzip -q '{ZIP_INPUT_DIR}' -d '/content/ESRGAN'

## <h1>PASO 5. Selección del modelo y escalamiento frame a frame
<h4>Configure el código y ejecute la celda para mejorar la escala de imágenes / fotogramas de video




In [ ]:
%cd /content/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth','models/PPON_G.pth','models/PPON_D.pth']  
device = torch.device(Choose_device) 


test_img_folder = '/content/ESRGAN/input_frames/*'  #@param{type:"string"}

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)

/content/ESRGAN
Model path models/RRDB_ESRGAN_x4.pth. 
Testing...
1 00858000
2 00847001
3 00944001
4 00987001
5 01051000
6 00930000
7 01014001
8 00959001
9 01018001
10 00953001


## <h1>PASO 6. ZIP final con los frames escalados.

In [ ]:
 #@markdown  Directorio del ZIP final en Drive
 ZIP_INPUT_DIR = '/content/drive/My Drive/ESRGAN_DAIN_3_OUT.zip' #@param{type:"string"}

!zip -r '{ZIP_INPUT_DIR}' /content/ESRGAN/results